<a href="https://colab.research.google.com/github/Sanskar-16/Ml-Foundation/blob/master/exercises/Exercise%204%20-%20Handling%20Complex%20Images/Exercise%204-Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [10]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy') > DESIRED_ACCURACY):   
          print("\nReached 99.9% accuracy, so stopping training!!")   
          self.model.stop_training = True

callbacks = myCallback()

--2021-09-04 16:44:31--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.145.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.145.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.03s   

2021-09-04 16:44:31 (89.1 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [11]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001), 
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 78400)            

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s/',
    target_size=(300,300),
    batch_size=10,
    class_mode='binary'
)

Found 80 images belonging to 2 classes.


In [14]:
history = model.fit(
    train_generator,
    epochs=20,
    steps_per_epoch=5,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/20
5/5 [==============================] - 1s 80ms/step - loss: 14.4947 - accuracy: 0.4600
Epoch 2/20
5/5 [==============================] - 0s 74ms/step - loss: 1.0010 - accuracy: 0.6400
Epoch 3/20
5/5 [==============================] - 0s 66ms/step - loss: 0.6121 - accuracy: 0.6600
Epoch 4/20
5/5 [==============================] - 0s 66ms/step - loss: 0.4233 - accuracy: 0.7800
Epoch 5/20
5/5 [==============================] - 0s 67ms/step - loss: 0.1662 - accuracy: 0.9400
Epoch 6/20
5/5 [==============================] - 0s 67ms/step - loss: 0.0878 - accuracy: 0.9800
Epoch 7/20
5/5 [==============================] - 0s 72ms/step - loss: 0.0491 - accuracy: 1.0000

Reached 99.9% accuracy, so stopping training!!
